# IBM Capstone Project: The Battle of the Neighborhoods

### Open new coffee shop in Athens, Greece

By- Shivani Chitre



# 1] Introduction/Business Problem


Clearly define a problem or an idea of your choice, where you would need to leverage the Foursquare location data to solve or execute. Remember that data science problems always target an audience and are meant to help a group of stakeholders solve a problem, so make sure that you explicitly describe your audience and why they would care about your problem.

### Background:

There are so many cafe locators such as google, yelp, zomato including FourSquare, that will tell you all about all the cafes in a nearby area when you do a quick search. Cafes are loved by a variety of diverse populations. They make a good hangout spot or even for a quick professional meeting.

Especially in a tourist area, cafes are a popular destination to try something local or get fast refreshment from all the traveling. This is where I come in from a business professional's perspective.


Imagine the following scenario:
You would like to invest in a coffee shop in Athens, Greece to start your own business. where would you open it up? There is so much competition already out there.
I would start by researching some geographical and statistical data about the Country and the city. Then I will examine the area of interest and its population, neighborhoods.



### Project Idea

The Question/Problem is to find a solution to where to open up a coffee shop that does not have too much competition but has a good amount of tourists/locals. There are lots of cafes in Athens we will try to detect locations that are not already crowded with coffee stores or cafes


My idea for the Capstone Project is to show how to get venue and location data from FourSquare and analyze that data to come up with potential locations to open up a cafe. A high-level approach is as follows:


1. The business investors pick a city location [for my project I will use athens]
2. The foursquare website is scrapped for the top cafes in the city
3. From this list of top cafes, the list is augmented with additional geographical data
4. Using this additional geographical data the top nearby cafes are selected
5. The number of cafes of all neighborhoods is obtained
6. A map is presented to the user showing all the cafes in the vicinity


### Target Audience

The coffee shop will be for all ages mostly Tourists, working professionals, bloggers, and locals. Age group-wise- teenagers, young adults, or even the older population.

# 2] Data

In this section, I will describe the data used to solve the problem as described previously

In this project, I will determine which place is good for opening a coffee shop  in Athens, Greece.

1. I will convert address data into their equivalent latitude and longitude values.
2. For Athens neighborhood data, I will use wikipedia , https://en.wikipedia.org/wiki/Category:Districts of Greece
3. I will use the Foursquare API to explore Athens neighborhoods and to get venues in neighborhoods.
4. I will use the Foursquare API to get venue ratings and likes in neighborhoods.
5. I will use the k-means clustering and Agglomerative algorithms to complete clustering task
6. I will use the Folium library to visualize the neighborhoods, venues , clusters in Athens


## Table of Contents
- Download and Explore Neigborhood Dataset
- Load and Analyze Venues of Neighborhoods in Athens
- Cluster Neighborhoods using kmean Clustering
- Discussion
- Conclusion

### 1. Download and Explore Neigborhood Dataset

In [1]:
#install libraries 
!pip install geopy    
!pip install folium   
!pip install geocoder

     |████████████████████████████████| 102kB 9.8MB/s ta 0:00:011
     |████████████████████████████████| 102kB 9.1MB/s ta 0:00:011


In [2]:
#import libraries 
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import matplotlib.pyplot as plt # for graphical usage 

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library
from folium import plugins
from folium.plugins import HeatMap

# main documentation page: http://beautiful-soup-4.readthedocs.io/en/latest/
# how to use the BeautifulSoup package: https://www.youtube.com/watch?v=ng2o98k983k video
from bs4 import BeautifulSoup 
import pandas as pd
import requests

print('Libraries imported.')

Libraries imported.


### First get the geocoordinates of Athens, Greece

In [3]:
# get coordinates of Athens
Athens_address = 'Athens, Greece'

geolocator = Nominatim(user_agent="Athens_explorer")
location = geolocator.geocode(Athens_address)
latitude = location.latitude
longitude = location.longitude
santorini_center = [latitude, longitude ]
print('The geograpical coordinate of {} are {}, {}.'.format(Athens_address, latitude, longitude))

The geograpical coordinate of Athens, Greece are 37.9839412, 23.7283052.


#### Now I will retrieve neighborhood data of Athens

In [4]:

# Read Greece regions data 
url = "https://en.wikipedia.org/wiki/Category:Geography_of_Greece_by_region"
source = requests.get(url).text
soup =  (source,'lxml')

neighborhoodList = []

# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text.replace(', Greece',''))
    
df_neighborhood = pd.DataFrame({"Neighborhood": neighborhoodList})
print("There are {} regions around {}".format(df_neighborhood.shape[0], Athens_address))

There are 16 regions around Athens, Greece


#### Get more information about population and regions

In [10]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_aad96d17175149e399bac06e3b9fbb4f = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='5rmKF8rGLRvoMQ19-XnmsXutwPQfj9yFEOu7sUBufkH_',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_aad96d17175149e399bac06e3b9fbb4f.get_object(Bucket='ibmcapstoneprojectfinalgreece-donotdelete-pr-l5pgi5ikvvycdx',Key='greece region.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head(10)


,Regional unit,Population (2011)
0,Central Athens,"1,029,520"
1,North Athens,"592,490"
2,South Athens,"529,826"
3,West Athens,"489,675"
4,piraeus,"448,997"
5,East Attica,"502,348"
6,West Attica,"160,927"


In [11]:

df.shape

(7, 2)

#### Get geographical coordinates of neighborhoods

In [12]:

body = client_aad96d17175149e399bac06e3b9fbb4f.get_object(Bucket='ibmcapstoneprojectfinalgreece-donotdelete-pr-l5pgi5ikvvycdx',Key='coordinates - Sheet1.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_neighborhood = pd.read_csv(body)
df_neighborhood.head(10)


,Neighborhood,Longitude,Latitude
0,Pagkrati,37.968056,23.743889
1,Ambelokipi,37.989722,23.764722
2,Goudi,37.988333,23.775833
3,Exarcheia,37.986111,23.734722
4,Patissia,38.033333,23.733333
5,lissia,37.976944,23.756944
6,Petralona,37.967222,23.709722
7,Plaka,37.972222,23.730556
8,Anafiotika,37.972222,23.727778
9,Koukaki,37.962500,23.720833


#### Create Athens map with neighborhoods

In [16]:
#create map of Athens neighborhoods using latitude and longitude values
map_Athens= folium.Map(location=[latitude, longitude], zoom_start=11)
for Latitude, Longitude, Neighborhood in zip(
        df_neighborhood['Latitude'], 
       df_neighborhood['Longitude'], 
       df_neighborhood['Neighborhood']):
    label = '{}, {}'.format(Neighborhood, Latitude, Longitude)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [Latitude, Longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Athens)  

map_Athens

## 2. Load Venues and Analyze Neighborhoods in Athens

Now I will Foursquare API for getting venues of neighborhoods

In [17]:

# Set up Foursquare
CLIENT_ID = 'UT5K4OUN04YYO1TOP2P2APTA4KVTNYGQD1BHN3NZZN3EK5VB' # your Foursquare ID
CLIENT_SECRET = '441JSRAV4NXTXO0WCXWPPZWC5GJCKQMMTYCJ01AKDTYL1K0C' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UT5K4OUN04YYO1TOP2P2APTA4KVTNYGQD1BHN3NZZN3EK5VB
CLIENT_SECRET:441JSRAV4NXTXO0WCXWPPZWC5GJCKQMMTYCJ01AKDTYL1K0C


In [18]:
search_query = 'Cafe'
radius = 50000
print(search_query + ' .... OK!')

Cafe .... OK!


In [19]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=UT5K4OUN04YYO1TOP2P2APTA4KVTNYGQD1BHN3NZZN3EK5VB&client_secret=441JSRAV4NXTXO0WCXWPPZWC5GJCKQMMTYCJ01AKDTYL1K0C&ll=37.9839412,23.7283052&v=20180605&query=Cafe&radius=50000&limit=100'

In [20]:
results = requests.get(url).json()

In [21]:
# assign relevant part of JSON to venues
cafe = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(cafe)
dataframe.head(100)

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",False,55acce63498e8f5c02258184,Πλ. Ομονοίας,GR,NaN,Ελλάδα,NaN,91,"[Πλ. Ομονοίας, Ελλάδα]","[{'label': 'display', 'lat': 37.98470100400789...",37.984701,23.727920,Μεταξουργείο,NaN,NaN,Hondos Cafè,v-1592661792,NaN
1,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",False,4cced373ba0a5481e8114b59,Πλατεία Ομονοίας 18,GR,Αθήνα,Ελλάδα,NaN,6,"[Πλατεία Ομονοίας 18, Αθήνα, Αττική, Ελλάδα]","[{'label': 'display', 'lat': 37.983981, 'lng':...",37.983981,23.728250,NaN,NaN,Αττική,Break Cafe,v-1592661792,NaN
2,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",False,4d12435b12916dcb6e98d48a,Μάγερ 18,GR,Αθήνα,Ελλάδα,NaN,455,"[Μάγερ 18, 104 38 Αθήνα, Αττική, Ελλάδα]","[{'label': 'display', 'lat': 37.98744093096235...",37.987441,23.725612,NaN,104 38,Αττική,Café Concerto,v-1592661792,NaN
3,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",False,4d357b8b5017a09337fb639b,Καραγιώργη Σερβίας 1,GR,Αθήνα,Ελλάδα,Πλατεία Συντάγματος 1,986,"[Καραγιώργη Σερβίας 1 (Πλατεία Συντάγματος 1),...","[{'label': 'display', 'lat': 37.97619286340874...",37.976193,23.733760,NaN,105 63,Αττική,Public Café,v-1592661792,82454942
4,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",False,4d0dfa695c46a093806511b4,Πεσματζόγλου 5,GR,Αθήνα,Ελλάδα,στο Αίθριο της Στοάς Βιβλίου,355,[Πεσματζόγλου 5 (στο Αίθριο της Στοάς Βιβλίου)...,"[{'label': 'display', 'lat': 37.98155940698001...",37.981559,23.731002,NaN,105 64,Αττική,Polis Art Cafe,v-1592661792,NaN
5,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",False,4ba35bcdf964a520303738e3,Θεμιστοκλέους 70,GR,Αθήνα,Ελλάδα,NaN,606,"[Θεμιστοκλέους 70, 114 73 Αθήνα, Αττική, Ελλάδα]","[{'label': 'display', 'lat': 37.98629653662064...",37.986297,23.734536,NaN,114 73,Αττική,Διπλό Cafe,v-1592661792,NaN
6,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",False,4b154927f964a52051aa23e3,Βουτάδων 44,GR,Αθήνα,Ελλάδα,NaN,1586,"[Βουτάδων 44, 118 54 Αθήνα, Αττική, Ελλάδα]","[{'label': 'display', 'lat': 37.97890971708671...",37.978910,23.711391,Γκάζι,118 54,Αττική,Del Sol Cafe,v-1592661792,NaN
7,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",False,56532388498e701dbfa89193,Αιόλου 77,GR,Αθήνα,Ελλάδα,NaN,473,"[Αιόλου 77, 105 51 Αθήνα, Αττική, Ελλάδα]","[{'label': 'display', 'lat': 37.97969743946321...",37.979697,23.728648,NaN,105 51,Αττική,Cafe 77,v-1592661792,NaN
8,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",False,4db6f90dcda1c57c828e9748,Ακαδημίας 95α,GR,Αθήνα,Ελλάδα,NaN,307,"[Ακαδημίας 95α, 106 77 Αθήνα, Αττική, Ελλάδα]","[{'label': 'display', 'lat': 37.98499438734644...",37.984994,23.731540,NaN,106 77,Αττική,Del Viale Café,v-1592661792,NaN
9,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",False,4d50d4e5d7eaa143d6766c0f,Σταδίου 40,GR,Αθήνα,Ελλάδα,NaN,465,"[Σταδίου 40, 106 79 Αθήνα, Αττική, Ελλάδα]","[{'label': 'display', 'lat': 37.98059179914281...",37.980592,23.731479,NaN,106 79,Αττική,Cafe Μαζί,v-1592661792,NaN


In [27]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_cafe = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
# filter the category for each row
dataframe_cafe['categories'] = dataframe_cafe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_cafe.columns = [column.split('.')[-1] for column in dataframe_cafe.columns]

dataframe_cafe.head(100)


,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Hondos Cafè,Café,Πλ. Ομονοίας,GR,NaN,Ελλάδα,NaN,91,"[Πλ. Ομονοίας, Ελλάδα]","[{'label': 'display', 'lat': 37.98470100400789...",37.984701,23.727920,Μεταξουργείο,NaN,NaN,55acce63498e8f5c02258184
1,Break Cafe,Café,Πλατεία Ομονοίας 18,GR,Αθήνα,Ελλάδα,NaN,6,"[Πλατεία Ομονοίας 18, Αθήνα, Αττική, Ελλάδα]","[{'label': 'display', 'lat': 37.983981, 'lng':...",37.983981,23.728250,NaN,NaN,Αττική,4cced373ba0a5481e8114b59
2,Café Concerto,Café,Μάγερ 18,GR,Αθήνα,Ελλάδα,NaN,455,"[Μάγερ 18, 104 38 Αθήνα, Αττική, Ελλάδα]","[{'label': 'display', 'lat': 37.98744093096235...",37.987441,23.725612,NaN,104 38,Αττική,4d12435b12916dcb6e98d48a
3,Public Café,Café,Καραγιώργη Σερβίας 1,GR,Αθήνα,Ελλάδα,Πλατεία Συντάγματος 1,986,"[Καραγιώργη Σερβίας 1 (Πλατεία Συντάγματος 1),...","[{'label': 'display', 'lat': 37.97619286340874...",37.976193,23.733760,NaN,105 63,Αττική,4d357b8b5017a09337fb639b
4,Polis Art Cafe,Café,Πεσματζόγλου 5,GR,Αθήνα,Ελλάδα,στο Αίθριο της Στοάς Βιβλίου,355,[Πεσματζόγλου 5 (στο Αίθριο της Στοάς Βιβλίου)...,"[{'label': 'display', 'lat': 37.98155940698001...",37.981559,23.731002,NaN,105 64,Αττική,4d0dfa695c46a093806511b4
5,Διπλό Cafe,Café,Θεμιστοκλέους 70,GR,Αθήνα,Ελλάδα,NaN,606,"[Θεμιστοκλέους 70, 114 73 Αθήνα, Αττική, Ελλάδα]","[{'label': 'display', 'lat': 37.98629653662064...",37.986297,23.734536,NaN,114 73,Αττική,4ba35bcdf964a520303738e3
6,Del Sol Cafe,Bar,Βουτάδων 44,GR,Αθήνα,Ελλάδα,NaN,1586,"[Βουτάδων 44, 118 54 Αθήνα, Αττική, Ελλάδα]","[{'label': 'display', 'lat': 37.97890971708671...",37.978910,23.711391,Γκάζι,118 54,Αττική,4b154927f964a52051aa23e3
7,Cafe 77,Café,Αιόλου 77,GR,Αθήνα,Ελλάδα,NaN,473,"[Αιόλου 77, 105 51 Αθήνα, Αττική, Ελλάδα]","[{'label': 'display', 'lat': 37.97969743946321...",37.979697,23.728648,NaN,105 51,Αττική,56532388498e701dbfa89193
8,Del Viale Café,Café,Ακαδημίας 95α,GR,Αθήνα,Ελλάδα,NaN,307,"[Ακαδημίας 95α, 106 77 Αθήνα, Αττική, Ελλάδα]","[{'label': 'display', 'lat': 37.98499438734644...",37.984994,23.731540,NaN,106 77,Αττική,4db6f90dcda1c57c828e9748
9,Cafe Μαζί,Café,Σταδίου 40,GR,Αθήνα,Ελλάδα,NaN,465,"[Σταδίου 40, 106 79 Αθήνα, Αττική, Ελλάδα]","[{'label': 'display', 'lat': 37.98059179914281...",37.980592,23.731479,NaN,106 79,Αττική,4d50d4e5d7eaa143d6766c0f


Filter cafes out of all venues,
get the total count

In [28]:
print('Total number of cafes:', len(dataframe_cafe['id'].unique()))



Total number of cafes: 50


In [29]:
# get counts of cafes in each Neighborhood


df_cafe_counts =  dataframe_cafe.groupby(['city']).count().rename(columns={"categories": "cafeCount"})[['cafeCount']]


df_cafe_counts.head(100)


,cafeCount
city,
Αθήνα,43
Καλλιθέας,1
Κυψέλη,1
Παγκράτι,2


In [60]:
#find neighborhoods that does not have any cafe 
nocafeList = list(set(dataframe_cafe['city']) - set(dataframe_cafe['city']))



#if exists , append neighborhoods without any restaurant to df_rest_counts
if nocafeList != []:
    df_cafe_counts = df_cafe_counts.append (pd.DataFrame( {'Neighborhood' : nocafeList  , 'cafeCount': [0] * len(nocafeList) } ).set_index('Neighborhood'))

df_cafe_counts.reset_index(inplace=True)

#####
# get counts of cafe in each Neighborhood
df_cafe_counts =  dataframe_cafe.groupby(['city']).count().rename(columns={"categories": "cafeCount"})[['cafeCount']]

#find neighborhoods that does not have any restaurant 
nocafeList = list(set(dataframe_cafe['city']) - set(dataframe_cafe['city']))

#if exists , append neighborhoods without any restaurant to df_rest_counts
if nocafeList != []:
    df_cafe_counts = df_cafe_counts.append (pd.DataFrame( {'city' : nocafeList  , 'cafeCount': [0] * len(nocafeList) } ).set_index('Neighborhood'))

df_cafe_counts.reset_index(inplace=True)
df_cafe_counts= df_cafe_counts.merge(df_cafe_counts).set_index('city')
df_cafe_counts= df_cafe_counts.sort_values(by=['cafeCount'],ascending =False)



print('{} neighborhoods do not have any cafes'.format(len(nocafeList)))

0 neighborhoods do not have any cafes


 From the data, we can see that the central Athens neighborhood has  most cafes. Kypseli and Pangrati does not have a lot of coffee shops, but are also popular areas in Athens in the central region as well. There are 9 neighborhoods in Athens. we can choose one of these to open a cafe. I will try to open a cafe in the most populated area which gets a lot of tourists/locals..

combining the datasets

In [31]:
df1= df
df2= df_neighborhood
df3= pd.merge(df1, df2, left_index=True, right_index=True)
df3.head(10)

,Regional unit,Population (2011),Neighborhood,Longitude,Latitude
0,Central Athens,"1,029,520",Pagkrati,37.968056,23.743889
1,North Athens,"592,490",Ambelokipi,37.989722,23.764722
2,South Athens,"529,826",Goudi,37.988333,23.775833
3,West Athens,"489,675",Exarcheia,37.986111,23.734722
4,piraeus,"448,997",Patissia,38.033333,23.733333
5,East Attica,"502,348",lissia,37.976944,23.756944
6,West Attica,"160,927",Petralona,37.967222,23.709722



Show restaurants on head map

Let's create a map showing heatmap / density of restaurants and try to extract some meaningful info from that. Also, let's show borders of Greece on our map and a few circles indicating distance of 2km, 4km, 6km and 10km from Athens center

Red markers are cafes

In [32]:
athens_center = [longitude, latitude]
map_athens= folium.Map(location=athens_center, zoom_start=12)
df = dataframe_cafe[dataframe_cafe['city'].isin(nocafeList)]

folium.TileLayer('cartodbpositron').add_to(map_athens) 
HeatMap(dataframe_cafe[['lat','lng']]).add_to(map_athens)
folium.Marker(athens_center).add_to(map_athens)
folium.Circle(athens_center, radius=2000, fill=False, color='white').add_to(map_athens)
folium.Circle(athens_center, radius=4000, fill=False, color='white').add_to(map_athens)
folium.Circle(athens_center, radius=6000, fill=False, color='white').add_to(map_athens)
folium.Circle(athens_center, radius=10000, fill=False, color='black').add_to(map_athens)
for lat, lon, neig, name in zip(dataframe_cafe['lat'], dataframe_cafe['lng'], dataframe_cafe['city'], dataframe_cafe['name']):
    label = folium.Popup(str(name) + ' - ' + str(neig), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_athens)
       
map_athens

From heat map , we can see that there are

Too many cafes in the immediate central athens vicinity.
Enough cafes between the main city towards the museum.
However, There are not many cafes near Pangrati neighborhood.
cafes are marked with red circle.
cafes are located pretty much in all directions from the main city includingSouth, North , East, betweenfew miles of the center.
There are not enough cafes outside of the man city area. 
Pangrati is a neighborhood in Central Athens, Greece with an estimated 35,173 residents.Pangrati has attracted artists from all over Greece, who arrive in the city of Athens to educate themselves and to seek inspiration from its vast pool of artistic resources and galleries. Poets, novelists, writers, painters, composers, musicians made Pangrati one of the most important artistic hubs of Athens.


# 3. Cluster Neigborhoods using Kmean Clustering

In [41]:
# one hot encoding
onehot = pd.get_dummies(dataframe_cafe[['categories']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
onehot['name'] = dataframe_cafe['name']

venues_grouped = onehot.groupby(["name"]).sum().reset_index()

# move neighborhood column to the first column and filter only restaurant columns 
fixed_columns =['name'] + list(dataframe_cafe['categories'].unique())
venues_grouped = venues_grouped[fixed_columns]

print("{}  cafe venue categories are shown in {} columns as below".format(venues_grouped.shape[0],venues_grouped.shape[1]-1))
venues_grouped.head(50)

50  cafe venue categories are shown in 10 columns as below


,name,Café,Bar,American Restaurant,Meze Restaurant,Greek Restaurant,Cafeteria,Restaurant,Performing Arts Venue,Dessert Shop,Italian Restaurant
0,Aegli Café,1,0,0,0,0,0,0,0,0,0
1,Alsos All Day Cafe Bar,1,0,0,0,0,0,0,0,0,0
2,Bello Cafe,1,0,0,0,0,0,0,0,0,0
3,Break Cafe,1,0,0,0,0,0,0,0,0,0
4,Cafe & Restaurant at Acropolis Museum (Καφέ & ...,1,0,0,0,0,0,0,0,0,0
5,Cafe (Ζαχαροπλαστείο Εύη),0,0,0,0,0,0,0,0,1,0
6,Cafe 77,1,0,0,0,0,0,0,0,0,0
7,Cafe Avissinia (Καφέ Αβυσσινία),1,0,0,0,0,0,0,0,0,0
8,Cafe Del Arte (Έντεχνον),1,0,0,0,0,0,0,0,0,0
9,Cafe Delli,1,0,0,0,0,0,0,0,0,0


In [40]:
len(venues_grouped[venues_grouped["Café"] > 0])

36

#### Create a new DataFrame for cafe data only

In [42]:

athens_cafe = venues_grouped[["name","Café"]]

In [44]:
athens_cafe.head(10)

,name,Café
0,Aegli Café,1
1,Alsos All Day Cafe Bar,1
2,Bello Cafe,1
3,Break Cafe,1
4,Cafe & Restaurant at Acropolis Museum (Καφέ & ...,1
5,Cafe (Ζαχαροπλαστείο Εύη),0
6,Cafe 77,1
7,Cafe Avissinia (Καφέ Αβυσσινία),1
8,Cafe Del Arte (Έντεχνον),1
9,Cafe Delli,1


Cluster Neighborhoods

Run k-means to cluster the cafes in Athens by neighborhoods

In [46]:
# set number of clusters
kclusters = 3

at_clustering = athens_cafe.drop(["name"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(at_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return_n_iter=True)


array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0], dtype=int32)

In [47]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
at_merged = athens_cafe.copy()

# add clustering labels
at_merged["Cluster Labels"] = kmeans.labels_

In [51]:
at_merged.rename(columns={"name": "name"}, inplace=True)
at_merged.head(10)

,name,Café,Cluster Labels
0,Aegli Café,1,0
1,Alsos All Day Cafe Bar,1,0
2,Bello Cafe,1,0
3,Break Cafe,1,0
4,Cafe & Restaurant at Acropolis Museum (Καφέ & ...,1,0
5,Cafe (Ζαχαροπλαστείο Εύη),0,1
6,Cafe 77,1,0
7,Cafe Avissinia (Καφέ Αβυσσινία),1,0
8,Cafe Del Arte (Έντεχνον),1,0
9,Cafe Delli,1,0


In [52]:
# merge to add latitude/longitude for each neighborhood
at_merged = at_merged.join(dataframe_cafe.set_index("name"), on="name")

print(at_merged.shape)
at_merged.head() # check the last columns!

(50, 18)


,name,Café,Cluster Labels,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Aegli Café,1,0,Café,Λεωφ. Βασιλίσσης Όλγας,GR,Αθήνα,Ελλάδα,στην Αίγλη Ζαππείου,1636,"[Λεωφ. Βασιλίσσης Όλγας (στην Αίγλη Ζαππείου),...","[{'label': 'display', 'lat': 37.97103493454318...",37.971035,23.737231,NaN,105 57,Αττική,4faf79cde4b085f6b2ff763c
1,Alsos All Day Cafe Bar,1,0,Café,Ευελπίδων & Μουστοξύδη,GR,Αθήνα,Ελλάδα,Πεδίον Άρεως,1601,"[Ευελπίδων & Μουστοξύδη (Πεδίον Άρεως), Αθήνα,...","[{'label': 'display', 'lat': 37.99482185141755...",37.994822,23.740248,NaN,NaN,Αττική,4db58b5f6a23c31a032c49fa
2,Bello Cafe,1,0,Café,Ευαγγελιστρίας 22,GR,Αθήνα,Ελλάδα,Περικλέους,750,"[Ευαγγελιστρίας 22 (Περικλέους), 105 60 Αθήνα,...","[{'label': 'display', 'lat': 37.97727258436337...",37.977273,23.729549,NaN,105 60,Αττική,51778dcd39506782cdc2f82e
3,Break Cafe,1,0,Café,Πλατεία Ομονοίας 18,GR,Αθήνα,Ελλάδα,NaN,6,"[Πλατεία Ομονοίας 18, Αθήνα, Αττική, Ελλάδα]","[{'label': 'display', 'lat': 37.983981, 'lng':...",37.983981,23.728250,NaN,NaN,Αττική,4cced373ba0a5481e8114b59
4,Cafe & Restaurant at Acropolis Museum (Καφέ & ...,1,0,Café,Διονυσίου Αρεοπαγίτου 15,GR,Αθήνα,Ελλάδα,στο Μουσείο Ακρόπολης,1685,[Διονυσίου Αρεοπαγίτου 15 (στο Μουσείο Ακρόπολ...,"[{'label': 'display', 'lat': 37.96880160758317...",37.968802,23.728242,NaN,117 42,Αττική,4cf27ecc8333224b7786108e


In [53]:
# sort the results by Cluster Labels
print(at_merged.shape)
at_merged.sort_values(["Cluster Labels"], inplace=True)
at_merged

(50, 18)


,name,Café,Cluster Labels,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Aegli Café,1,0,Café,Λεωφ. Βασιλίσσης Όλγας,GR,Αθήνα,Ελλάδα,στην Αίγλη Ζαππείου,1636,"[Λεωφ. Βασιλίσσης Όλγας (στην Αίγλη Ζαππείου),...","[{'label': 'display', 'lat': 37.97103493454318...",37.971035,23.737231,NaN,105 57,Αττική,4faf79cde4b085f6b2ff763c
25,Ianos Café,1,0,Café,Σταδίου 24,GR,Αθήνα,Ελλάδα,NaN,651,"[Σταδίου 24, 105 64 Αθήνα, Αττική, Ελλάδα]","[{'label': 'display', 'lat': 37.97909666310944...",37.979097,23.732474,NaN,105 64,Αττική,4adcdadef964a520ff5721e3
27,JOJO Cafe,1,0,Café,Φωκίωνος Νέγρη 18-20,GR,Κυψέλη,Ελλάδα,NaN,1982,"[Φωκίωνος Νέγρη 18-20, 112 57 Κυψέλη, Αττική, ...","[{'label': 'display', 'lat': 38.00084040230515...",38.000840,23.735444,NaN,112 57,Αττική,4c766e1dff1fb60c79d5f8a7
28,La familia cafe,1,0,Café,Πατησιων 105,GR,Αθήνα,Ελλάδα,NaN,1390,"[Πατησιων 105, Αθήνα, Αττική, Ελλάδα]","[{'label': 'display', 'lat': 37.99616793952725...",37.996168,23.731542,NaN,NaN,Αττική,51531bc6e4b0276ef3322112
29,Melina Café (Μελίνα Café),1,0,Café,Λυσίου 22,GR,Αθήνα,Ελλάδα,NaN,1185,"[Λυσίου 22, 105 56 Αθήνα, Αττική, Ελλάδα]","[{'label': 'display', 'lat': 37.97329465919812...",37.973295,23.728730,NaN,105 56,Αττική,4b8e829ef964a520222633e3
30,Merlin Cafe,1,0,Café,Μέρλιν 1,GR,Αθήνα,Ελλάδα,Λεωφ. Βασιλίσσης Σοφίας 9,1252,"[Μέρλιν 1 (Λεωφ. Βασιλίσσης Σοφίας 9), 106 75 ...","[{'label': 'display', 'lat': 37.97600347500009...",37.976003,23.738417,NaN,106 75,Αττική,4d80b7ca39c588bfac83073f
33,Pasmal Cafe,1,0,Café,Μητροπόλεως 8,GR,Αθήνα,Ελλάδα,NaN,1060,"[Μητροπόλεως 8, 105 63 Αθήνα, Αττική, Ελλάδα]","[{'label': 'display', 'lat': 37.97518383137719...",37.975184,23.733055,NaN,105 63,Αττική,51077f27e4b0bf4054552b13
35,Plaka's Gate Cafe,1,0,Café,Διονυσίου Αρεοπαγίτου 4,GR,Αθήνα,Ελλάδα,NaN,1607,"[Διονυσίου Αρεοπαγίτου 4, 105 58 Αθήνα, Αττική...","[{'label': 'display', 'lat': 37.96964133424408...",37.969641,23.730890,NaN,105 58,Αττική,4c2dc8fb7d85a593531c53f3
48,Πόλις Cafe,1,0,Café,NaN,GR,Αθήνα,Ελλάδα,NaN,825,"[Αθήνα, Αττική, Ελλάδα]","[{'label': 'display', 'lat': 37.99072366623438...",37.990724,23.732099,NaN,NaN,Αττική,53be2e91498ed32fb4ce91da
36,Polis Art Cafe,1,0,Café,Πεσματζόγλου 5,GR,Αθήνα,Ελλάδα,στο Αίθριο της Στοάς Βιβλίου,355,[Πεσματζόγλου 5 (στο Αίθριο της Στοάς Βιβλίου)...,"[{'label': 'display', 'lat': 37.98155940698001...",37.981559,23.731002,NaN,105 64,Αττική,4d0dfa695c46a093806511b4


#### Finally, let's visualize the resulting clusters

In [55]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(at_merged['lat'], at_merged['lng'], at_merged['name'], at_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [56]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### Examine Clusters

#### cluster 0

In [57]:

at_merged.loc[at_merged['Cluster Labels'] == 0]

,name,Café,Cluster Labels,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Aegli Café,1,0,Café,Λεωφ. Βασιλίσσης Όλγας,GR,Αθήνα,Ελλάδα,στην Αίγλη Ζαππείου,1636,"[Λεωφ. Βασιλίσσης Όλγας (στην Αίγλη Ζαππείου),...","[{'label': 'display', 'lat': 37.97103493454318...",37.971035,23.737231,NaN,105 57,Αττική,4faf79cde4b085f6b2ff763c
25,Ianos Café,1,0,Café,Σταδίου 24,GR,Αθήνα,Ελλάδα,NaN,651,"[Σταδίου 24, 105 64 Αθήνα, Αττική, Ελλάδα]","[{'label': 'display', 'lat': 37.97909666310944...",37.979097,23.732474,NaN,105 64,Αττική,4adcdadef964a520ff5721e3
27,JOJO Cafe,1,0,Café,Φωκίωνος Νέγρη 18-20,GR,Κυψέλη,Ελλάδα,NaN,1982,"[Φωκίωνος Νέγρη 18-20, 112 57 Κυψέλη, Αττική, ...","[{'label': 'display', 'lat': 38.00084040230515...",38.000840,23.735444,NaN,112 57,Αττική,4c766e1dff1fb60c79d5f8a7
28,La familia cafe,1,0,Café,Πατησιων 105,GR,Αθήνα,Ελλάδα,NaN,1390,"[Πατησιων 105, Αθήνα, Αττική, Ελλάδα]","[{'label': 'display', 'lat': 37.99616793952725...",37.996168,23.731542,NaN,NaN,Αττική,51531bc6e4b0276ef3322112
29,Melina Café (Μελίνα Café),1,0,Café,Λυσίου 22,GR,Αθήνα,Ελλάδα,NaN,1185,"[Λυσίου 22, 105 56 Αθήνα, Αττική, Ελλάδα]","[{'label': 'display', 'lat': 37.97329465919812...",37.973295,23.728730,NaN,105 56,Αττική,4b8e829ef964a520222633e3
30,Merlin Cafe,1,0,Café,Μέρλιν 1,GR,Αθήνα,Ελλάδα,Λεωφ. Βασιλίσσης Σοφίας 9,1252,"[Μέρλιν 1 (Λεωφ. Βασιλίσσης Σοφίας 9), 106 75 ...","[{'label': 'display', 'lat': 37.97600347500009...",37.976003,23.738417,NaN,106 75,Αττική,4d80b7ca39c588bfac83073f
33,Pasmal Cafe,1,0,Café,Μητροπόλεως 8,GR,Αθήνα,Ελλάδα,NaN,1060,"[Μητροπόλεως 8, 105 63 Αθήνα, Αττική, Ελλάδα]","[{'label': 'display', 'lat': 37.97518383137719...",37.975184,23.733055,NaN,105 63,Αττική,51077f27e4b0bf4054552b13
35,Plaka's Gate Cafe,1,0,Café,Διονυσίου Αρεοπαγίτου 4,GR,Αθήνα,Ελλάδα,NaN,1607,"[Διονυσίου Αρεοπαγίτου 4, 105 58 Αθήνα, Αττική...","[{'label': 'display', 'lat': 37.96964133424408...",37.969641,23.730890,NaN,105 58,Αττική,4c2dc8fb7d85a593531c53f3
48,Πόλις Cafe,1,0,Café,NaN,GR,Αθήνα,Ελλάδα,NaN,825,"[Αθήνα, Αττική, Ελλάδα]","[{'label': 'display', 'lat': 37.99072366623438...",37.990724,23.732099,NaN,NaN,Αττική,53be2e91498ed32fb4ce91da
36,Polis Art Cafe,1,0,Café,Πεσματζόγλου 5,GR,Αθήνα,Ελλάδα,στο Αίθριο της Στοάς Βιβλίου,355,[Πεσματζόγλου 5 (στο Αίθριο της Στοάς Βιβλίου)...,"[{'label': 'display', 'lat': 37.98155940698001...",37.981559,23.731002,NaN,105 64,Αττική,4d0dfa695c46a093806511b4


#### cluster 1

In [58]:
at_merged.loc[at_merged['Cluster Labels'] == 1]

,name,Café,Cluster Labels,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
5,Cafe (Ζαχαροπλαστείο Εύη),0,1,Dessert Shop,NaN,GR,NaN,Ελλάδα,NaN,2550,[Ελλάδα],"[{'label': 'display', 'lat': 37.97821996218016...",37.978220,23.756447,NaN,NaN,NaN,5297092b11d24b01834b24ed
42,millennium cafe,0,1,Bar,NaN,GR,Αθήνα,Ελλάδα,NaN,769,"[Αθήνα, Αττική, Ελλάδα]","[{'label': 'display', 'lat': 37.98597445715895...",37.985974,23.719926,NaN,NaN,Αττική,5357d2cd498ee8116ffc3b70
41,Union Café,0,1,Cafeteria,Διδότου,GR,Αθήνα,Ελλάδα,Μασσαλίας 22,839,"[Διδότου (Μασσαλίας 22), 106 80 Αθήνα, Αττική,...","[{'label': 'display', 'lat': 37.98182524607237...",37.981825,23.737487,NaN,106 80,Αττική,4f11b765e4b067f66fe3c8e8
18,Del Sol Cafe,0,1,Bar,Βουτάδων 44,GR,Αθήνα,Ελλάδα,NaN,1586,"[Βουτάδων 44, 118 54 Αθήνα, Αττική, Ελλάδα]","[{'label': 'display', 'lat': 37.97890971708671...",37.978910,23.711391,Γκάζι,118 54,Αττική,4b154927f964a52051aa23e3
34,Pesto Trattoria & Cafe,0,1,Italian Restaurant,NaN,GR,Αθήνα,Ελλάδα,NaN,164,"[Αθήνα, Αττική, Ελλάδα]","[{'label': 'display', 'lat': 37.98391379962638...",37.983914,23.730176,NaN,NaN,Αττική,501a5c01e4b07bd245a45e5b
32,Musique Cafe and more...,0,1,Bar,Αρριανού 37,GR,Αθήνα,Ελλάδα,Αρχελάου,2029,"[Αρριανού 37 (Αρχελάου), 116 35 Αθήνα, Αττική,...","[{'label': 'display', 'lat': 37.97184506164029...",37.971845,23.745608,NaN,116 35,Αττική,4f382d1ce4b0fc67aa066f17
31,Metro Café,0,1,Greek Restaurant,Πλατεία Μοναστηρακίου 5,GR,Αθήνα,Ελλάδα,NaN,878,"[Πλατεία Μοναστηρακίου 5, 105 55 Αθήνα, Αττική...","[{'label': 'display', 'lat': 37.97625816821012...",37.976258,23.726038,NaN,105 55,Αττική,4c3f75a4ff711b8da2f20e05
26,In Town Cafe,0,1,Greek Restaurant,Μητροπόλεως 12,GR,Αθήνα,Ελλάδα,NaN,1003,"[Μητροπόλεως 12, 105 63 Αθήνα, Αττική, Ελλάδα]","[{'label': 'display', 'lat': 37.97538225706555...",37.975382,23.731906,NaN,105 63,Αττική,4e6a0f95d4c01ce01762e1ed
17,Cycladic Cafe,0,1,Restaurant,Νεοφύτου Δούκα 4,GR,Αθήνα,Ελλάδα,στο Μουσείο Κυκλαδικής Τέχνης,1530,[Νεοφύτου Δούκα 4 (στο Μουσείο Κυκλαδικής Τέχν...,"[{'label': 'display', 'lat': 37.97585094240011...",37.975851,23.742402,NaN,106 74,Αττική,4f045a51823162ada697b68b
23,Hard Rock Cafë,0,1,Bar,Atina,GR,Αθήνα,Ελλάδα,NaN,68,"[Atina, Αθήνα, Αττική, Ελλάδα]","[{'label': 'display', 'lat': 37.98381, 'lng': ...",37.983810,23.727540,NaN,NaN,Αττική,584a72f98b0ebb3d3b65d101


# 4. Results/ Discussion

Observations:
Most of the cafes are concentrated in the central area of Athens city, with the highest number in cluster 0 and moderate number in cluster 1. On the other hand, cluster 1 has a very low number of cafes in the neighborhoods and they are spread out. This represents a great opportunity and high potential areas to open new coffee shops in the areas I suggested before as there is very little to no competition. Meanwhile, cafes in cluster 0 are likely suffering from intense competition due to oversupply and high concentration of other similar locations. From another perspective, this also shows that the crowding of cafes mostly happened in the central area of the city which was the most popular tourist attraction so it makes sense.  The neighboring areas a few kilometers away from the central city has fewer stores yet has a high potential. Therefore, this project recommends property developers to capitalize on these findings to open a new coffee shop in neighborhoods in cluster 1 with little to no competition. Kypseli and Pangrati do not have a lot of coffee shops but are also popular areas in Athens in the central region as well. Lastly, property developers are advised to avoid neighborhoods in cluster 0 (Main Athens)  which already have a high concentration of cafes, restaurants, and are facing adverse competition. 


# 5. Conclusion

In this project, we have gone through the process of identifying the business problem, specifying the data required, extracting and preparing the data, performing machine learning by clustering the data into 2 clusters based on their similarities, and lastly providing recommendations to the relevant stakeholders i.e. property developers and investors regarding the best locations to open a new cafe. To answer the business question that was raised in the introduction section, the answer proposed by this project is: The neighborhoods in cluster 1 are the most preferred locations to open a new cafe. The findings of this project will help the relevant stakeholders to capitalize on the opportunities on high potential locations while avoiding overcrowded areas in their decisions to open a new cafe in Athens, Greece. 		